In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Capstone/

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/Capstone/

/content/gdrive/MyDrive/Capstone


In [ ]:
!pip install swifter

     |████████████████████████████████| 634kB 12.1MB/s 
     |████████████████████████████████| 296kB 25.5MB/s 
     |████████████████████████████████| 583kB 25.3MB/s 
     |████████████████████████████████| 112kB 46.8MB/s 
     |████████████████████████████████| 17.2MB 348kB/s 
     |████████████████████████████████| 48.5MB 114kB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 204kB 33.2MB/s 
     |████████████████████████████████| 133kB 47.3MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 1.3MB 41.7MB/s 
     |████████████████████████████████| 3.1MB 19.2MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 296kB 51.3MB/s 
     |████████████████████████████████| 143kB 48.7MB/s 
  Created wheel for swifter: filename=swifter-1.0.7-cp37-none-any.whl size=13980 sha256=99330290facb8cc13eb4416

In [ ]:
!pip install pytorch_pretrained_bert pytorch-nlp

     |████████████████████████████████| 133kB 10.7MB/s 
     |████████████████████████████████| 92kB 6.7MB/s 
     |████████████████████████████████| 133kB 17.9MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 7.4MB 12.6MB/s 
ERROR: botocore 1.20.57 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!pip install torchtext

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 10.3MB/s 
     |████████████████████████████████| 901kB 46.3MB/s 
     |████████████████████████████████| 3.3MB 41.5MB/s 


## Data Preparation

In [ ]:
# data analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# data preprocessing libraries
import re
from datetime import datetime
import spacy
import nltk
from nltk import word_tokenize, sent_tokenize
from spacy_preprocessor import SpacyPreprocessor
from sklearn.model_selection import train_test_split

# other
from tqdm import tqdm, tqdm_notebook, tqdm_pandas

tqdm.pandas()
tqdm_notebook().pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import random

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token, eos_token, pad_token, unk_token)
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

[CLS] [SEP] [PAD] [UNK]
101 102 0 100


In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

512


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField()

In [ ]:
fields = {'text': ('text',TEXT), 'target': ('label', LABEL)}

In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path = 'data', 
    train = 'hitop_train.csv',
    validation = 'hitop_val.csv',
    test = 'hitop_test.csv',
    format ='csv',
    fields = fields,
    skip_header = False
)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 56446
Number of validation examples: 14112
Number of testing examples: 17640


In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    batch_size = BATCH_SIZE, 
    device = device)

In [ ]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn

class HITOPBERT(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
        
        _, hidden = self.rnn(embedded)
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        
        output = self.out(hidden)
        
        
        return output, hidden

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 10
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.4

model = HITOPBERT(bert, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 110,667,530 trainable parameters
The model has 1,185,290 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)[0]

        loss = criterion(predictions, batch.label)
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()

    with torch.no_grad():

        for batch in iterator:

            predictions = model(batch.text)[0]
            
            loss = criterion(predictions, batch.label)

            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'HITOP_RNN.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 7m 17s
	Train Loss: 1.093 | Train Acc: 64.37%
	 Val. Loss: 0.817 |  Val. Acc: 73.86%
Epoch: 02 | Epoch Time: 7m 18s
	Train Loss: 0.794 | Train Acc: 74.59%
	 Val. Loss: 0.757 |  Val. Acc: 76.08%
Epoch: 03 | Epoch Time: 7m 17s
	Train Loss: 0.713 | Train Acc: 76.97%
	 Val. Loss: 0.764 |  Val. Acc: 75.51%
Epoch: 04 | Epoch Time: 7m 18s
	Train Loss: 0.661 | Train Acc: 78.77%
	 Val. Loss: 0.738 |  Val. Acc: 77.03%
Epoch: 05 | Epoch Time: 7m 17s
	Train Loss: 0.617 | Train Acc: 79.88%
	 Val. Loss: 0.736 |  Val. Acc: 77.16%


In [ ]:
model.load_state_dict(torch.load('HITOP_RNN.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.744 | Test Acc: 77.14%


In [ ]:
def predict_hitop(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor)[0])
    return prediction #, model(tensor)[1]

In [ ]:
hitop_test = pd.read_csv('data/hitop_test.csv')

In [ ]:
y = hitop_test.text.progress_apply(lambda x: predict_hitop(model, tokenizer, x))